================================
###### CEK NORMALISASI SKOR CT
###### Data siswa dari kategori Siaga, Penggalang, dan Penegak diolah dengan normalisasi karena jumlah soal, bobot, dan tingkat kesulitannya beda. Skor mentahnya juga punya rentang yang beda-beda, bahkan ada yang minus karena sistem penalti, jadi nggak bisa dibandingin langsung. 
###### Biar adil, semua skor dinormalisasi pakai metode Min–Max per kategori. Hasilnya, nilai tiap siswa ada di rentang 0 sampai 1: yang paling rendah jadi 0, paling tinggi jadi 1, sisanya di antaranya. 
###### Setelah itu, data dari ketiga kategori digabung jadi satu dataset dengan struktur kolom yang disamain. Nilai CT yang dipakai buat analisis selanjutnya adalah nilai hasil normalisasi.
================================

In [1]:
import pandas as pd

siaga = pd.read_csv('files/Siaga_updated.csv', sep=';', encoding='latin1')
penggalang = pd.read_csv('files/Penggalang_updated.csv', sep=';', encoding='latin1')
penegak = pd.read_csv('files/Penegak_updated.csv', sep=';', encoding='latin1')

print("Siaga - Nilai Min:", siaga['Nilai'].min(), "Nilai Max:", siaga['Nilai'].max())
print("Penggalang - Nilai Min:", penggalang['Nilai'].min(), "Nilai Max:", penggalang['Nilai'].max())
print("Penegak - Nilai Min:", penegak['Nilai'].min(), "Nilai Max:", penegak['Nilai'].max())


Siaga - Nilai Min: -19.31 Nilai Max: 81.25
Penggalang - Nilai Min: -13.33 Nilai Max: 82.33
Penegak - Nilai Min: -3.56 Nilai Max: 54.59


In [2]:
# ambil semua kolom soal (S1, S2, dst)
kolom_soal_siaga = [col for col in siaga.columns if col.strip().startswith('S ')]
kolom_soal_penggalang = [col for col in penggalang.columns if col.strip().startswith('S ')]
kolom_soal_penegak = [col for col in penegak.columns if col.strip().startswith('S ')]

print("SIAGA - MIN semua soal:", siaga[kolom_soal_siaga].min().min())
print("SIAGA - MAX semua soal:", siaga[kolom_soal_siaga].max().max())

print("PENGGALANG - MIN semua soal:", penggalang[kolom_soal_penggalang].min().min())
print("PENGGALANG - MAX semua soal:", penggalang[kolom_soal_penggalang].max().max())

print("PENEGAK - MIN semua soal:", penegak[kolom_soal_penegak].min().min())
print("PENEGAK - MAX semua soal:", penegak[kolom_soal_penegak].max().max())


SIAGA - MIN semua soal: -2.08
SIAGA - MAX semua soal: 8.33
PENGGALANG - MIN semua soal: -1.67
PENGGALANG - MAX semua soal: 6.67
PENEGAK - MIN semua soal: -1.67
PENEGAK - MAX semua soal: 6.67


In [3]:
siaga['CT_norm'] = (
    (siaga['Nilai'] - siaga['Nilai'].min()) /
    (siaga['Nilai'].max() - siaga['Nilai'].min())
)

penggalang['CT_norm'] = (
    (penggalang['Nilai'] - penggalang['Nilai'].min()) /
    (penggalang['Nilai'].max() - penggalang['Nilai'].min())
)

penegak['CT_norm'] = (
    (penegak['Nilai'] - penegak['Nilai'].min()) /
    (penegak['Nilai'].max() - penegak['Nilai'].min())
)   

print("Siaga:", siaga['CT_norm'].min(), siaga['CT_norm'].max())
print("Penegak:", penegak['CT_norm'].min(), penegak['CT_norm'].max())
print("Penggalang:", penggalang['CT_norm'].min(), penggalang['CT_norm'].max())


Siaga: 0.0 1.0
Penegak: 0.0 1.0
Penggalang: 0.0 1.0


In [4]:
import pandas as pd
import re

def standardize_soal_columns(df):
    rename_dict = {}
    for col in df.columns:
        # tangkap: S  1  8 33, S1, S 1, dll
        match = re.match(r'^S\s*(\d+)', str(col))
        if match:
            rename_dict[col] = f"S{match.group(1)}"
    return df.rename(columns=rename_dict)

siaga = standardize_soal_columns(siaga)
penegak = standardize_soal_columns(penegak)
penggalang = standardize_soal_columns(penggalang)

soal_cols = [f"S{i}" for i in range(1, 16)]

def ensure_all_soal(df):
    for col in soal_cols:
        if col not in df.columns:
            df[col] = pd.NA
    return df

siaga = ensure_all_soal(siaga)
penegak = ensure_all_soal(penegak)
penggalang = ensure_all_soal(penggalang)

siaga['Kategori'] = 'Siaga'
penegak['Kategori'] = 'Penegak'
penggalang['Kategori'] = 'Penggalang'

df_siswa = pd.concat(
    [siaga, penegak, penggalang],
    ignore_index=True
)

if 'CT_norm' in df_siswa.columns:
    df_siswa['CT_norm'] = df_siswa['CT_norm'].round(3)

df_siswa.to_csv(
    'files/data_CTnorm_siswa.csv',
    index=False,
    sep=';',
    encoding='utf-8-sig'
)

print("✅ Data siswa berhasil digabung & disimpan!")


✅ Data siswa berhasil digabung & disimpan!


In [5]:
print("Total baris :", len(df_siswa))
print("Total kolom :", len(df_siswa.columns))
print(df_siswa.head())

Total baris : 1678
Total kolom : 35
       Id                     Nama               Alamat surel  Kelas  \
0  318534    Aaron Nathanael Yules  aar2490907@bebras2024.com      4   
1  318530  Abraham Giovani Soediro  abr2421711@bebras2024.com      6   
2  288718    Abyatar Putra Nugraha  aby2426848@bebras2024.com      6   
3  288860         Adelle Dwi Alexa  ade2478482@bebras2024.com      6   
4  318500  Adrea Naryama Bellvania  adr2424916@bebras2024.com      5   

  JenisKelamin                   SekolahNama SekolahKotaKabupaten  \
0            L         SD Kristen Banjarsari            Surakarta   
1            L         SD Kristen Banjarsari            Surakarta   
2            L  SD Kristen Manahan Surakarta            Surakarta   
3            P  SD Kristen Manahan Surakarta            Surakarta   
4            P         SD Kristen Banjarsari            Surakarta   

  SekolahJenjang     Provinsi       CreatedDate  ...    S8    S9   S10   S11  \
0             SD  Jawa Tengah  04/11

In [6]:
kolom_diambil = ['Nama', 'Kelas', 'Kategori', 'Nilai', 'CT_norm']

df_siswa[kolom_diambil].groupby('Kategori').head(3)


,Nama,Kelas,Kategori,Nilai,CT_norm
0,Aaron Nathanael Yules,4,Siaga,12.92,0.321
1,Abraham Giovani Soediro,6,Siaga,3.61,0.228
2,Abyatar Putra Nugraha,6,Siaga,23.75,0.428
385,Abel Jenesi Natalia,11,Penegak,23.33,0.462
386,Abigail Sekar Ayu Bening,11,Penegak,24.44,0.482
387,Adinda Saschia,11,Penegak,-0.50,0.053
450,AALEYAH HELGA DENYAN,8,Penggalang,11.33,0.258
451,ABELIZA NUR RACHMADANA,8,Penggalang,44.00,0.599
452,ABHIPRAYA RADITYA RAFA AFIANTO,8,Penggalang,16.67,0.314
